<h1>Project 2 George Stigler

Our populations of interest for this project is food deserts and non-food deserts in the United States. When comparing the average diets between the two types of areas, it is evident that those in non-food deserts are more likely to meet the Recommended Dietary Allowances (RDAs) for various nutrients, whereas those in food deserts might not meet these requirements, leading to nutritional deficiencies or excesses in unhealthy nutrients like saturated fats and added sugars.

Deliverables included in this file:

- Dietary Reference Intakes
- Data on Prices for Different Foods
- Nutritional Content of Food
- Solution - what is the minimum food cost for food deserts across America?
- What is the total cost to feed the entirety of the US?


<h2>[A] Dietary Reference Intakes

In [29]:
#data for function below was found on: https://health.gov/sites/default/files/2019-09/Appendix-E3-1-Table-A4.pdf

In [30]:
import pandas as pd

# Define the function to get DRIs based on the provided information
def get_dri(age, sex):
    """
    Returns a pandas.Series of Dietary Reference Intakes (DRIs) for the given age and sex.

    Parameters:
    age (int): The age of the person.
    sex (str): The sex of the person. Expected values: 'male' or 'female'.

    Returns:
    pandas.Series: A series of DRIs for various nutrients.
    """

    # DRI values based on the provided information
    dri_values = {
        'Protein_g': None,
        'Carbohydrate_g': None,
        'Carbohydrate_percent_kcal': None,
        'Dietary_Fiber_g': None,
        'Total_fat_percent_kcal': None,
        'Saturated_fat_percent_kcal': None
    }
    
    # Assigning values based on age and sex
    if age >= 1 and age <= 3:
        dri_values['Protein_g'] = 13 if sex == 'female' else 13
        dri_values['Dietary_Fiber_g'] = 14
    elif age >= 4 and age <= 8:
        dri_values['Protein_g'] = 19 if sex == 'female' else 19
        dri_values['Dietary_Fiber_g'] = 16.8 if sex == 'female' else 19.6
    elif age >= 9 and age <= 13:
        dri_values['Protein_g'] = 34 if sex == 'female' else 34
        dri_values['Dietary_Fiber_g'] = 22.4 if sex == 'female' else 25.2
    elif age >= 14 and age <= 18:
        dri_values['Protein_g'] = 46 if sex == 'female' else 52
        dri_values['Dietary_Fiber_g'] = 25.2 if sex == 'female' else 30.8
    elif age >= 19 and age <= 30:
        dri_values['Protein_g'] = 46 if sex == 'female' else 56
        dri_values['Dietary_Fiber_g'] = 28 if sex == 'female' else 33.6
    elif age >= 31 and age <= 50:
        dri_values['Protein_g'] = 46 if sex == 'female' else 56
        dri_values['Dietary_Fiber_g'] = 25.2 if sex == 'female' else 30.8
    elif age >= 51:
        dri_values['Protein_g'] = 46 if sex == 'female' else 56
        dri_values['Dietary_Fiber_g'] = 22.4 if sex == 'female' else 28
    
    # Carbohydrate and fat values are consistent across all age groups
    dri_values['Carbohydrate_g'] = 130  # Consistent for all age groups and sexes
    dri_values['Carbohydrate_percent_kcal'] = (45, 65)  # Consistent for all age groups and sexes
    dri_values['Total_fat_percent_kcal'] = (20, 35)  # Consistent for all age groups and sexes
    dri_values['Saturated_fat_percent_kcal'] = 10  # Consistent for all age groups and sexes

    # Convert the dictionary to a pandas Series
    dri_series = pd.Series(dri_values, name=f'DRIs for {age}-year-old {sex}')

    return dri_series
    
#EXAMPLE
six_year_old_female = get_dri(6, 'female')
six_year_old_female

Protein_g                           19
Carbohydrate_g                     130
Carbohydrate_percent_kcal     (45, 65)
Dietary_Fiber_g                   16.8
Total_fat_percent_kcal        (20, 35)
Saturated_fat_percent_kcal          10
Name: DRIs for 6-year-old female, dtype: object

In [31]:
apikey = "Z4ehkkMbO1DlcukePiecqfxU55sRATGnDJHItl5B"

In [32]:
from  scipy.optimize import linprog as lp
import numpy as np
import warnings

In [34]:
SHEETs = [# Stigler's foods, modern prices
          ("https://docs.google.com/spreadsheet/ccc?key=1ObK5N_5aVXzVHE7ZXWBg0kQvPS3k1enRwsUjhytwh5A","Stigler Table B (2022 Prices)"),
         ]

In [35]:
import pandas as pd
%pip install eep153_tools
%pip install gnupg
from eep153_tools.sheets import read_sheets

df = read_sheets(SHEETs[0][0])[SHEETs[0][1]]
df

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Key available for students@eep153.iam.gserviceaccount.com.


,Food,Quantity,Units,Price,Date,Location,FDC
0,Wheat Flour,80.0,oz,3.79,[2022-2-28],Safeway,2432946
1,Wheat Cereal,16.4,oz,3.49,[2022-2-28],Safeway,2343964
2,Corn Meal,80.0,oz,4.49,[2022-2-28],Safeway,2079814
3,Rolled Oats,42.0,oz,3.99,[2022-2-28],Safeway,2478125
4,Evaporated Milk,12.0,oz,1.99,[2022-2-28],Safeway,1889123
5,Cabbage,1.0,lbs,1.29,[2022-2-28],Safeway,169975
6,Potatoes,1.0,lbs,1.49,[2022-2-28],Safeway,576920
7,Spinach,1.0,oz,0.25,[2022-2-28],Safeway,168462
8,Sweet Potatoes,1.0,lbs,1.99,[2022-2-28],Safeway,2427040
9,Navy Beans,1.0,lbs,3.49,[2022-2-28],Safeway,1859097


In [36]:
%pip install fooddatacentral
import fooddatacentral as fdc
import warnings

D = {}
count = 0
for food in  df.Food.tolist():
    try:
        FDC = df.loc[df.Food==food,:].FDC[count]
        count+=1
        D[food] = fdc.nutrients(apikey,FDC).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))        

FoodNutrients = pd.DataFrame(D,dtype=float)

Note: you may need to restart the kernel to use updated packages.


In [38]:
# Convert food quantities to FDC units
df['FDC Quantity'] = df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Now may want to filter df by time or place--need to get a unique set of food names.
df['FDC Price'] = df['Price']/df['FDC Quantity']

df.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Prices = df.groupby('Food',sort=False)['FDC Price'].min()
Prices

Food
Wheat Flour        0.16711039473613717 / hectogram
Wheat Cereal        0.7506471170977783 / hectogram
Corn Meal          0.19797511144202004 / hectogram
Rolled Oats         0.3351026385210139 / hectogram
Evaporated Milk     0.5849598689972085 / hectogram
Cabbage            0.28439631821849204 / hectogram
Potatoes           0.32848877065546755 / hectogram
Spinach             0.8818490487395102 / hectogram
Sweet Potatoes      0.4387199017479063 / hectogram
Navy Beans          0.7694132950252227 / hectogram
Sugar              0.20888799342017147 / hectogram
Beets              0.28439631821849204 / hectogram
Milk (Whole)       0.13974701569746056 / deciliter
Liver (Beef)        1.3205689504874165 / hectogram
Name: FDC Price, dtype: object

<h2>[A] Nutritional Content of Food (For non-Food Deserts)

You can change the value in group to find the nutritional value of the non-food desert diet for that population. You can also see the avg price of that diet.

In [39]:
group = 'M 19-30'
tol = 1e-6

result = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

Cost of diet for M 19-30 is $9.20 per day.


Diet (in 100s of grams or milliliters):
Rolled Oats      1.820657
Spinach          6.997015
Milk (Whole)    10.233686
Liver (Beef)     0.748359
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          2407.785776          2400.0
Protein                           91.470404            56.0
Fiber, total dietary              33.600000            33.6
Folate, DFE                     1574.444808           400.0
Calcium, Ca                     2049.863704          1000.0
Carbohydrate, by difference      198.586576           130.0
Iron, Fe                          31.732150             8.0
Magnesium, Mg                    689.038827           400.0
Niacin                            16.000000            16.0
Phosphorus, P                   1666.070726           700.0
Potassium, K         